In [1]:
from pdf_helper import get_pdf_data
from config import *
import os

from langchain import PromptTemplate

In [2]:
full_text = get_pdf_data("VARO.pdf")

In [3]:
from langchain_openai import AzureChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
len(full_text)

244282

In [5]:
import os

os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY

In [6]:
AZURE_OPENAI_DEPLOYMENT_GPT_4O_ID

'gpt-4o-mini'

In [7]:
llm = AzureChatOpenAI(deployment_name = AZURE_OPENAI_DEPLOYMENT_GPT_4O_ID,
                      model="gpt-4o-mini")

In [8]:
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000267C2E87210>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000267C2FDD450>, root_client=<openai.lib.azure.AzureOpenAI object at 0x00000267C3080E50>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x00000267C2EB0390>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://westusgenai.openai.azure.com/', deployment_name='gpt-4o-mini', openai_api_version='2023-05-15', openai_api_type='azure')

In [9]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=10000, chunk_overlap=500)

docs = text_splitter.create_documents([full_text])

In [10]:
len(docs)

26

In [11]:
summary = ""
pages_per_call = 2
delay = 60
import time

In [12]:
summary

''

In [13]:
map_prompt = """
Please summarize the following text:
{text}
"""

reduce_prompt = """
Please summarize the following text:
{text}
"""
map_prompt_template = PromptTemplate(template=map_prompt, 
                                     input_variables=["text"])
reduce_prompt_template = PromptTemplate(template=reduce_prompt,
                                        input_variables=["text"])

In [14]:
for i in range(0, len(docs), pages_per_call):
        batch_docs = docs[i:i+pages_per_call]
        chain = load_summarize_chain(llm, chain_type="map_reduce", 
                                     map_prompt=map_prompt_template, 
                                     combine_prompt=reduce_prompt_template)
        try:
            summary += chain.run(batch_docs)
        except Exception as e:
            print(f"Error during summarization of pages {i+1}-{min(i+pages_per_call, len(docs))}: {e}")
            continue

        print(f"Summarized pages {i+1}-{min(i+pages_per_call, len(docs))}")
        # time.sleep(delay) # Wait to avoid rate limits   

C:\Users\Ambarish Ganguly\AppData\Local\Temp\ipykernel_25164\690631642.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary += chain.run(batch_docs)


Summarized pages 1-2
Summarized pages 3-4
Summarized pages 5-6
Summarized pages 7-8
Summarized pages 9-10
Summarized pages 11-12
Summarized pages 13-14
Summarized pages 15-16
Summarized pages 17-18
Summarized pages 19-20
Summarized pages 21-22
Summarized pages 23-24
Summarized pages 25-26


In [15]:
from pprint import pprint

In [16]:
pprint(summary)

("The VARO Energy ESG Report 2022 highlights the company's dedication to "
 'environmental, social, and governance (ESG) initiatives within its '
 'transformation strategy. It outlines proactive measures for reducing '
 'environmental impacts, enhancing product sustainability, and lowering carbon '
 'emissions, focusing on areas like biodiversity, waste management, health and '
 'safety, equity, inclusion, and community engagement. \n'
 '\n'
 'CEO Dev Sanyal emphasizes energy security amidst global challenges, '
 'particularly due to the Ukraine conflict. VARO aims for net-zero emissions '
 'by 2040 and a 40% reduction in scope 1 and 2 emissions by 2030, backed by a '
 '$3.5 billion investment plan from 2022 to 2026. The report stresses the '
 'integration of ESG principles in decision-making and prioritizes safety and '
 'diversity within its workforce.\n'
 '\n'
 'To address evolving market challenges, VARO seeks collaboration, skill '
 'development, and an inclusive culture, enhancin

In [17]:
summary

'The VARO Energy ESG Report 2022 highlights the company\'s dedication to environmental, social, and governance (ESG) initiatives within its transformation strategy. It outlines proactive measures for reducing environmental impacts, enhancing product sustainability, and lowering carbon emissions, focusing on areas like biodiversity, waste management, health and safety, equity, inclusion, and community engagement. \n\nCEO Dev Sanyal emphasizes energy security amidst global challenges, particularly due to the Ukraine conflict. VARO aims for net-zero emissions by 2040 and a 40% reduction in scope 1 and 2 emissions by 2030, backed by a $3.5 billion investment plan from 2022 to 2026. The report stresses the integration of ESG principles in decision-making and prioritizes safety and diversity within its workforce.\n\nTo address evolving market challenges, VARO seeks collaboration, skill development, and an inclusive culture, enhancing leadership diversity in the process. The company balances 